In [2]:
# Compared Luanna's allocation list with finished entry HJ allocation items 
# Created by : Jim,Shen on Feb.25.2023

# Steps:
# 1. Save allocation list on path:  'C:\Users\jjshe\Downloads\Wanek_Allocation_List.xlsx'
# 2. Download HJ allocation items on path:  'C:\Users\jjshe\Downloads\Allocated_List.xlsx'
# 3. Run this python file
# 4. Get compared report on path: '‪C:\Users\jishen\WN3_Allocation_Compared_Report.xlsx'


In [3]:
import pandas as pd
import pymsgbox

# loading destination into dict
data = pd.read_excel(r'C:\Users\luannguyen\Downloads\Wanek_Allocation_List.xlsx', sheet_name='Destination',dtype={'Destination':str,'Whs':str})
dict1 = dict(zip(data['Destination'],data['Whs']))
# print(dict1)

df = pd.read_excel(r'C:\Users\luannguyen\Downloads\Wanek_Allocation_List.xlsx',sheet_name=1,skiprows=1,
                   dtype={'Item':str,'Priority':str,'WHs':str,'Total':str,'Destination':str})

df['Due'] = pd.to_datetime(df['Due'],errors='coerce')
df['Due'] = df['Due'].dt.strftime('%m/%d/%Y')
df.set_index=['Item','Destination']
print(df)

# import excel file Allocated_List.xlsx into df2 then loaded into dictionary:
df2 = pd.read_excel(r'C:\Users\luannguyen\Downloads\Allocated_List.xlsx',sheet_name=0,usecols='B,D,H:Z',
                    dtype={'Production Item':str})
df2['*MFG Date'] = pd.to_datetime(df2['*MFG Date'],errors='coerce')
df2['*MFG Date'] = df2['*MFG Date'].dt.strftime('%m/%d/%Y')
df2 = df2[df2['*MFG Date']==df['Due'][0]]
df2.reset_index(inplace=True)
df2.drop(columns='index',inplace=True)

# Un-piovtTable
df2=pd.melt(df2,id_vars=['*MFG Date','Production Item'],var_name='Destination',value_name='AllocatedQty')
df2 = df2[df2['AllocatedQty'] != 0]
df2.reset_index(inplace=True)
df2.drop(columns='index',inplace=True)

# through dict1 to query destination whse for comparision with df1
df2['Whs'] = df2['Destination'].apply(lambda  x: dict1[x])   
df2.rename(columns={'*MFG Date':'Due','Production Item':'Item','Whs':'WHs'},inplace=True)
df2.set_index=['Item','WHs']


# df['Total'] =df['Total'].apply(pd.to_numeric,errors='coerce')
df['Total'] =df['Total'].astype(int)

#  comparing
df3 = pd.merge(df,df2,on = ['Item','WHs'],how='outer')
print(df3)
df3['AllocatedQty-PlannedQty(Total)'] = df3['AllocatedQty']-df3['Total']
df3 = df3[df3['AllocatedQty-PlannedQty(Total)'] != 0]
df3.to_excel(r'C:\Users\luannguyen\Downloads\WN3_Allocation_Compared_Report.xlsx',index=False)

pymsgbox.alert("Compared Finished!",'Notice:','OK')


            Due      Item Priority  WHs Total
0    04/01/2023   1000764        6  335     2
1    04/01/2023   1000764       18    5    26
2    04/01/2023   1000765        6  335     3
3    04/01/2023   1000765       15   42     2
4    04/01/2023   1000765       18    5    23
..          ...       ...      ...  ...   ...
274  04/01/2023   9900435        6  335     8
275  04/01/2023   9900438        6  335     9
276  04/01/2023  U2570547       16  ECR     3
277  04/01/2023  U5240219       16  ECR     6
278  04/01/2023  U8460415       18    5     1

[279 rows x 5 columns]
          Due_x     Item Priority  WHs  Total       Due_y Destination  \
0    04/01/2023  1000764        6  335    2.0  04/01/2023      Ashton   
1    04/01/2023  1000764       18    5   26.0         NaN         NaN   
2    04/01/2023  1000765        6  335    3.0  04/01/2023      Ashton   
3    04/01/2023  1000765       15   42    2.0  04/01/2023          42   
4    04/01/2023  1000765       18    5   23.0  04/01/2023  

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\luannguyen\\Downloads\\WN3_Allocation_Compared_Report.xlsx'